In [2]:
import pandas
import subprocess, os
import seaborn as sns
import ast
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import re
from sklearn import preprocessing as pre
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
from collections import OrderedDict
from operator import itemgetter
import itertools
from collections import Counter
import json
import os



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wzb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wzb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load the dataset
#nsf_dataset = pandas.read_csv('nsf-search-AR.csv')
#dataset = pandas.read_csv('newDATA-16Oct2020.csv')
dataset = pandas.read_csv('data-Mar11.csv', encoding = "ISO-8859-1")
alias = pandas.read_csv('term-bucketing.csv')
replace = pandas.read_csv('replacements-new.csv')
edges = pandas.read_csv('data_lab_test.csv')
ideas = pandas.read_csv('ideas_final2.csv')
#ideas_ = pandas.read_csv('ideas_scored_V2.csv')
baseline = pandas.read_csv('basline.csv')
lows = pandas.read_csv('lows.csv')
mids = pandas.read_csv('mids.csv')
highs = pandas.read_csv('highs.csv')
#ideas = pandas.read_csv('research_ideas.csv', encoding = "ISO-8859-1")


In [4]:
ideas

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Title,Description,Key Words,Terms,Categories,Stakeholders,Importance,...,TagsPlusAbstract,Sim Scores,Mean Sim,Stdev Sim,Max Score,Sim Scores (new),Acad Filtered Scores,Fict Filtered Scores,Acad Points,Score
0,0,0,0,Visualization of Data About or From Tools in A...,Combining live sensor data and tools with AR d...,"connected tools, manufacturing, AR display, in...","interactive systems and tools, Internet of Thi...","Business, Technology, Use Cases, Industries","Operators of manufacturing, repair, maintenanc...",AREA members with manufacturing facilities for...,...,visualization data tool ar display combining l...,"[0.007068058647219471, 0.20977597990017335, 0....",0.069014,0.042070,0.296288,"[0.007068058647219471, 0.20977597990017335, 0....","[0.20977597990017335, 0.09912355169777673, 0.0...","[0.20977597990017335, 0.040410219548672396, 0....",7075,3.44
1,1,1,1,AR for Material Management and/or Supply Chain...,Many industries track supply chains and juggle...,"logistics data processing, materials managemen...","interactive systems and tools, Internet of Thi...","Business, Technology, Use Cases, End Users","operators of manufacturing, repair, employees ...",AREA members with manufacturing facilities for...,...,ar material management supply chain flow work ...,"[0.006817976767927102, 0.06988174099523532, 0....",0.060330,0.033708,0.216651,"[0.006817976767927102, 0.06988174099523532, 0....","[0.06988174099523532, 0.07900191760728746, 0.0...","[0.06988174099523532, 0.03146423276656763, 0.0...",4788,2.33
2,2,2,2,Dynamic Distribution of Sensing and Processing...,Deploying enterprise AR solutions via head-mou...,"distributed computing architecture, body-worn ...","body sensor networks, personal area networks, ...","Display Devices, Standards, Business, Technology","HMD manufacturers, AR display device designers...",Employees in AREA member companies frequently ...,...,dynamic distribution sensing processing capaci...,"[0.013390898860077499, 0.047576539636666706, 0...",0.057300,0.031986,0.260657,"[0.013390898860077499, 0.047576539636666706, 0...","[0.08877646543133542, 0.0737144002922752, 0.07...","[0.047576539636666706, 0.08877646543133542, 0....",4097,1.99
3,3,3,3,Wearable vs Handheld vs Projection: Methods fo...,"Tablets, mobile phones and other handheld devi...","efficiency, handheld, projection AR, wearable ...","head worn displays, wearable computers,","Display Devices, Business","Operations leaders, financial management, OEM ...",Wearable AR displays provide frontline workers...,...,wearable v handheld v projection method quanti...,"[0.033671177087511106, 0.08104311856430285, 0....",0.063788,0.035677,0.246399,"[0.033671177087511106, 0.08104311856430285, 0....","[0.08104311856430285, 0.07448066853416974, 0.0...","[0.033671177087511106, 0.08104311856430285, 0....",5510,2.68
4,4,4,4,Impact of AR Delivery on People Living in Mult...,"The Multidimensional Poverty Index (MPI) ""meas...","United Nations, Sustainable Development Goals ...","social aspects, social and economic effects, s...","Business, End Users",Government officials and policymakers in World...,"The Multidimensional Poverty Index (MPI) ""meas...",...,impact ar delivery people living multidimensio...,"[0.010956530827323164, 0.06569176479038012, 0....",0.029010,0.018373,0.123466,"[0.010956530827323164, 0.06569176479038012, 0....","[0.06569176479038012, 0.09857682463539011, 0.0...","[0.06569176479038012, 0.037585230210463096, 0....",445,0.22
5,5,5,5,Video Passthrough vs. See-through Optics: Impa...,Head-Mounted AR displays (HMD) blend simulated...,"Video see-through, optical passthrough, field ...","head-up displays, three-dimensional displays, ...","Display Devices, Technology, End Users","Operations leaders, HMD designers, Safety & IT...",Most of the leading AR display component and s...,...,video passthrough v see optic impact display t...,"[0.01424284389398072, 0.07154493986180466, 0.0...",0.066330,0.041469,0.320431,"[0.014242843

In [5]:


import ast



In [30]:
topics = pandas.read_csv('research_topics.csv')

In [31]:
topics

,Unnamed: 0,Title,Description,Keywords,Categories,Stakeholders,Importance,Methdologies,Timeframes,Program,Miscellaneous,Author,History
0,0,Visualization of Data About or From Tools in A...,Combining live sensor data and tools with AR d...,"connected tools, manufacturing, AR display, in...","Business, Technology, Use Cases, Industries","Operators of manufacturing, repair, maintenanc...",- AREA members with manufacturing facilities f...,An experimental environment using a range of c...,Medium,This research topic is broad in scope. It can ...,Two AREA members with financial support from b...,Christine Perey,2021-08-31
1,1,AR for Material Management and/or Supply Chain...,Many industries track supply chains and juggle...,"logistics data processing, materials managemen...","Business, Technology, Use Cases, End User and ...","operators of manufacturing, repair, employees ...",- AREA members with manufacturing facilities f...,An experimental environment using a range of m...,Medium,This research topic can be combined with studi...,Three non-commercial AREA members with financi...,Christine Perey,2021-08-31
2,2,Dynamic Distribution of Sensing and Processing...,Deploying enterprise AR solutions via head-mou...,"distributed computing architecture, body-worn ...","Displays, Standards, Business, Technology","HMD manufacturers, AR display device designers...",- Employees in AREA member companies frequentl...,"Studies of wearable sensors, low-latency body ...",Long,This topic is at the intersection of multiple ...,Studies of optical and IMU sensor fusion for A...,"Peter Orban, Christine Perey",2021-08-31
3,3,Wearable vs Handheld vs Projection: Methods fo...,"Tablets, mobile phones, and other handheld dev...","efficiency, handheld, projection AR, wearable ...","Displays, Business","Operations leaders, financial management, OEM ...",- Wearable AR displays provide frontline worke...,The research will contribute to development of...,Near,This research can be combined with or extended...,The AREA RoI calculator is a starting point fo...,"Peter Orban, Christine Perey",2021-08-31
4,4,Impact of AR Delivery on People Living in Mult...,"The Multidimensional Poverty Index (MPI) ""meas...","United Nations, Sustainable Development Goals ...","Business, End User and User Experience",Government officials and policymakers in World...,- Because AR is a unique delivery format that ...,The proposed research would need to decide upo...,Near,This study could link closely with existing re...,"The Multidimensional Poverty Index (MPI) ""meas...",Jennifer Rogers,2021-08-31
5,5,Passthrough vs. See-through: Impacts of Displa...,Head-mounted AR displays (HMD) blend simulated...,"video see-through, video passthrough, field of...","Displays, Technology, End User and User Experi...","Operations leaders, HMD designers, Safety & IT...",- Most of the leading AR display component and...,Studies of human peripheral vision requirement...,Near,This research can be combined with or extended...,"Although quite dated by today's stamdards, one...","Peter Orban, Christine Perey",2021-08-31
6,6,New Power Sources for Wearable AR Displays,As the complexity and computational requiremen...,"power use, power consumption, power production...","Displays, Technology",All users of wearable AR display devices and t...,- Most of the leading AR display component and...,The research will be leveraging developments i...,Long,This research topic could be studied in differ...,"This field of research is in its infancy, but ...",Christine Perey,2021-08-31
7,7,Low Power Digital Signal Processors for Use in...,While the data about the user's context is cap...,"Low power, power use, power consumption, digit...","Displays, Technology","AR display designers, AR display manufacturers...",- Most of the leading AR display component and...,Semiconductor and DSP designers can collaborat...,Long,This research can be combined with fresh resea...,A 2018 https://www.qualcomm.com/media/document...,Christine Perey,20

In [25]:
dataset['Citation'][3704]

'Norman, M., Lee, G., Smith, R. T., & Billinqhurs, M. (2019). A Mixed Presence Collaborative Mixed Reality System. 2019 IEEE Conference on Virtual Reality and 3D User Interfaces (VR). doi:10.1109/vr.2019.8797966'

In [34]:
len(ideas)

46

In [35]:
len(topics)

47

In [ ]:
with open("sampleACAD.txt") as f:
    baselineAcad = f.readlines()
f.close()
with open("sampleFICT.txt") as f:
    baselineFict = f.readlines()
f.close()

In [ ]:
lengths = []
for i in range(0,len(baselineFict)):
    lengths.append(len(baselineFict[i]))
    if len(baselineAcad[i])==6:
        print(i)

In [ ]:
min(lengths)

In [ ]:
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
#new_words = ["br", "use", "award","project","research","using","new","based","nsf","develop","application"] 
#stop_words = stop_words.union(new_words)


#df.drop(columns=['cleanAbstract'])

for i in range(0, len(baselineAcad)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z0-9]', ' ', baselineAcad[i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    #text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    baselineAcad[i]=text
    
for i in range(0, len(baselineFict)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z0-9]', ' ', baselineFict[i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    #text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    baselineFict[i]=text
    

#https://scikit-learn.org/stable/auto_examples/semi_supervised/plot_label_propagation_digits_active_learning.html

In [ ]:
for i in range (0, len(edges)):
    for j in range(0,len(alias)):
        source_term = edges['Source'][i]
        target_term = edges['Target'][i]
        term = alias['LowLevel'][j]
        if source_term == term:
            edges['midlevel-source'][i]=alias['MidLevel'][j]
        if target_term == term:
            edges['midlevel-target'][i]=alias['MidLevel'][j]            

In [ ]:
term = []
for i in range (0, len(alias)):
    term.append(alias['MidLevel'][i])
term = list(set(term))
        

In [ ]:
for i in range (0, len(edges)):
    term1 = edges['midlevel-source'][i]
    term2 = edges['midlevel-target'][i]
    if term1 == term2:
        edges['Label'][i]=term1

In [ ]:
for i in range(0,len(term)):
    term_m = term[i]
    edges[term_m].replace(0, np.nan, inplace=True)

In [ ]:
for i in range (0, len(edges)):
    source = edges['midlevel-source'][i]
    target = edges['midlevel-target'][i]
    for j in range (0, len(term)):
        term_m = term[j]
        if source==term_m or target==term_m:
            edges[term_m][i]=1
#            print('yes!')
#        if target==term_m:
#            edges[term_m][i]=1
#            print('yes!')
        else:
            edges[term_m][i]=0
#    print(i)

In [ ]:
edges.to_csv("data_lab_test.csv")

In [36]:
ideas = pandas.read_csv('research_topics.csv', encoding = "ISO-8859-1")

In [37]:
def cosine_model(m, classes=None, **cos_kws):

    encode = pre.MultiLabelBinarizer(classes=classes)
    tags = encode.fit_transform(m)

    cos = cosine_similarity(tags.T.dot(tags))
    cos -= np.diag(cos.diagonal())
    return cos

vectorizer = TfidfVectorizer()
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [38]:
control_keywords = []
uncontrol_keywords =[]
classification_code = []

for i in range(0,len(dataset)):
 
    
    class_code = dataset['Classification code'][i]
    res = isinstance(class_code, str)
    if res == False:
        dataset['Classification code'][i] = "other"
        class_code = dataset['Classification code'][i]
    class_code = re.split(';',class_code)
    
       
    control_keys = dataset['Controlled/Subject terms'][i]
    res = isinstance(control_keys, str)
    if res == False:
        dataset['Controlled/Subject terms'][i] = "other"
        control_keys = dataset['Controlled/Subject terms'][i]
    #dataset['LowLevel'] = control_keys.lower()
    control_keys = re.split(';',control_keys)

    
    uncontrol_keys = dataset['Uncontrolled terms'][i]
    res = isinstance(uncontrol_keys, str)
    if res == False:
        dataset['Uncontrolled terms'][i] = "other"
        uncontrol_keys = dataset['Uncontrolled terms'][i]
    uncontrol_keys = re.split(';',uncontrol_keys)
  
    for j in range(0,len(control_keys)):
        control_keys[j] = control_keys[j].lower()

    for k in range(0,len(uncontrol_keys)):
        uncontrol_keys[k] = uncontrol_keys[k].lower()
        
    #for l in range(0,len(class_code)):
        #class_code[l] = class_code[l].lower()
        
    control_keywords.append(control_keys)
    uncontrol_keywords.append(uncontrol_keys)
    classification_code.append(class_code)


In [39]:
for i in range(0,len(dataset)):
    dataset['LowLevel'][i]=dataset['Controlled/Subject terms'][i].lower()

for i in range(0, len(dataset)):
    low=dataset['LowLevel'][i]
    terms=low.split(';')
    terms_to_delete =[]
    for l in range(0,len(terms)):
        term=terms[l]
        for k in range(0,len(replace)):
            check_term = replace['FIND'][k]
            replace_term = replace['REPLACE'][k]
            if check_term == term:
                if replace['REPLACE'][k]=="DELETE":
                    terms_to_delete.append(term)
                if replace['REPLACE'][k]!="DELETE":
                    terms[l]=replace_term

    for m in range(0, len(terms_to_delete)):
        terms.remove(terms_to_delete[m])
    
    dataset['LowLevel'][i]=terms



C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [40]:
all_mids = []
all_highs = []
for i in range(0,len(dataset)):
    mid=[]
    high=[]
    terms=dataset['LowLevel'][i]
    for k in range(0,len(terms)):
        for j in range(0,len(alias)):
            low = terms[k]
            if low == alias['LowLevel'][j]:
                mid.append(alias['MidLevel'][j])
                high.append(alias['HighLevel'][j]) 
    dataset['MidLevel'][i]=list(set(mid))
    all_mids.append(list(set(mid)))
    dataset['HighLevel'][i]=list(set(high))
    all_highs.append(list(set(high)))

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [41]:
concat = []
for i in range(0,len(all_mids)):
    mid_list = all_mids[i]
    if len(all_mids[i])==0:
        string_new = 'other'
    else:
        for j in range(0,len(mid_list)):
     
            if j==0:
                string_new = mid_list[j]
            else:
                string_new = string_new + ';' + mid_list[j]
    concat.append(string_new)
dataset['concat_test']=concat
dataset['midlvl']=concat    

In [42]:
#create new columns in database that are semicolon separated
llvl = []
for i in range(0,len(dataset)):
    low_list = str(dataset['LowLevel'][i])
#    mid_list = str(dataset['MidLevel'][i])
    low_list=re.sub("'","",low_list)
    low_list=re.sub(", ",";",low_list)
    low_list=re.sub('\[','',low_list)
    low_list=re.sub('\]','',low_list)
#    mid_list=re.sub("'","",mid_list)
#    mid_list=re.sub(", ",";",mid_list)
#    mid_list=re.sub('\[','',mid_list)
#    mid_list=re.sub('\]','',mid_list)
#    dataset['midlvl'][i]=mid_list
    llvl.append(low_list)
dataset['lowlvl']=llvl

In [43]:
llvl = []
counter=0
for i in range(0,len(dataset)):
    low = str(dataset['lowlvl'][i])
    if low.lower()=='':
        counter=counter+1
        dataset['lowlvl'][i]='other'
        
#print(counter)

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [44]:
combined_tags = []
skip_index=[]
#df.drop(columns=['combinedTags'])
for i in range (0, len(dataset)):
    
    if pandas.isna(dataset.loc[i, 'lowlvl']) == True:
        lowlevel_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'lowlvl']) == False: 
        lowlevel_terms = dataset['lowlvl'][i]
        
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == True:
        uncontrol_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == False:
        uncontrol_terms = dataset['Uncontrolled terms'][i]
    
    if pandas.isna(dataset.loc[i, 'Classification code']) == True:
        class_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'Classification code']) == False: 
        class_terms = dataset['Classification code'][i]
        
    if pandas.isna(dataset.loc[i, 'midlvl']) == True:
        midlevel_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'midlvl']) == False: 
        midlevel_terms = dataset['midlvl'][i]        
    
    
    lowlevel_terms = re.sub(' ', '_',lowlevel_terms)
    uncontrol_terms = re.sub(' ', '_',uncontrol_terms)
    class_terms = re.sub(' ', '_',class_terms)
    midlevel_terms = re.sub(' ', '_',midlevel_terms)
    
    #replace spaces with 
    lowlevel_terms = re.split(';', lowlevel_terms)
    uncontrol_terms = re.split(';', uncontrol_terms)
    class_terms = re.split(';', class_terms)
    midlevel_terms = re.split(';', midlevel_terms)
    
    text = str(lowlevel_terms) + str(uncontrol_terms) + str(class_terms) + str(midlevel_terms)
    #text = re.sub('.','_',text)
    text = re.sub('[^_a-zA-Z0-9]', ' ', text)
    text = text.lower()
    #text=re.sub("(\\d|\\W)+"," ",text)
    text = text.lstrip()
    text = text.rstrip()
    text = re.sub(' +', ' ',text)
    combined_tags.append(text)
    
dataset['combinedTagsNew'] = combined_tags
skip_index = list(set(skip_index))

In [45]:
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
#new_words = ["br", "use", "award","project","research","using","new","based","nsf","develop","application"] 
#stop_words = stop_words.union(new_words)

corpus = []
#df.drop(columns=['cleanAbstract'])

for i in range(0, len(dataset)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z0-9]', ' ', dataset['Abstract'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    #text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
    
dataset['cleanAbstract'] = corpus
#https://scikit-learn.org/stable/auto_examples/semi_supervised/plot_label_propagation_digits_active_learning.html

In [46]:
combined_tags = []
combined_tags1 = []
skip_index=[]
#df.drop(columns=['combinedTags'])
for i in range (0, len(dataset)):
    
    if pandas.isna(dataset.loc[i, 'lowlvl']) == True:
        lowlevel_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'lowlvl']) == False: 
        lowlevel_terms = dataset['lowlvl'][i]
        
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == True:
        uncontrol_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == False:
        uncontrol_terms = dataset['Uncontrolled terms'][i]    

 
    
    lowlevel_terms = re.sub(' ', '_',lowlevel_terms)
    #uncontrol_terms = re.sub(' ', '_',uncontrol_terms)
    #class_terms = re.sub(' ', '_',class_terms)
    #midlevel_terms = re.sub(' ', '_',midlevel_terms)
    
    #replace spaces with 
    lowlevel_terms = re.split(';', lowlevel_terms)
    uncontrol_terms = re.split(';', uncontrol_terms)
    #class_terms = re.split(';', class_terms)
    #midlevel_terms = re.split(';', midlevel_terms)
    
    text = str(lowlevel_terms) 
    #text = re.sub('.','_',text)
    text = re.sub('[^_a-zA-Z0-9]', ' ', text)
    text = text.lower()
    #text=re.sub("(\\d|\\W)+"," ",text)
    text = text.lstrip()
    text = text.rstrip()
    text = re.sub(' +', ' ',text)
    combined_tags.append(text)
    
    text1 = str(uncontrol_terms) 
    #text = re.sub('.','_',text)
    text1 = re.sub('[^_a-zA-Z0-9]', ' ', text1)
    text1 = text1.lower()
    #text=re.sub("(\\d|\\W)+"," ",text)
    text1 = text1.lstrip()
    text1 = text1.rstrip()
    text1 = re.sub(' +', ' ',text1)
    combined_tags1.append(text1)
    
    
    
dataset['ideaTags'] = combined_tags
dataset['ideaAbs'] = combined_tags1
skip_index = list(set(skip_index))

In [47]:
dataset['TagsPlusAbstract'] = dataset['combinedTagsNew'] + ' '+dataset['cleanAbstract'] 

In [48]:
dataset['IdeasTagsPlusAbstract'] = dataset['cleanAbstract'] + dataset['ideaTags'] + dataset['ideaAbs']

In [49]:
combined_tags = []
skip_index=[]
#df.drop(columns=['combinedTags'])
for i in range (0, len(dataset)):
    
    if pandas.isna(dataset.loc[i, 'lowlvl']) == True:
        lowlevel_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'lowlvl']) == False: 
        lowlevel_terms = dataset['lowlvl'][i]
        
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == True:
        uncontrol_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'Uncontrolled terms']) == False:
        uncontrol_terms = dataset['Uncontrolled terms'][i]
    
    if pandas.isna(dataset.loc[i, 'Classification code']) == True:
        class_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'Classification code']) == False: 
        class_terms = dataset['Classification code'][i]
        
    if pandas.isna(dataset.loc[i, 'midlvl']) == True:
        midlevel_terms = ''
        skip_index.append(i)
    if pandas.isna(dataset.loc[i, 'midlvl']) == False: 
        midlevel_terms = dataset['midlvl'][i]        
    
    
    lowlevel_terms = re.sub(' ', '_',lowlevel_terms)
    uncontrol_terms = re.sub(' ', '_',uncontrol_terms)
    class_terms = re.sub(' ', '_',class_terms)
    midlevel_terms = re.sub(' ', '_',midlevel_terms)
    
    #replace spaces with 
    lowlevel_terms = re.split(';', lowlevel_terms)
    uncontrol_terms = re.split(';', uncontrol_terms)
    class_terms = re.split(';', class_terms)
    midlevel_terms = re.split(';', midlevel_terms)
    
    text = str(lowlevel_terms) + str(uncontrol_terms) + str(class_terms) + str(midlevel_terms)
    #text = re.sub('.','_',text)
    text = re.sub('[^_a-zA-Z0-9]', ' ', text)
    text = text.lower()
    #text=re.sub("(\\d|\\W)+"," ",text)
    text = text.lstrip()
    text = text.rstrip()
    text = re.sub(' +', ' ',text)
    combined_tags.append(text)
    
dataset['combinedTagsNew'] = combined_tags
skip_index = list(set(skip_index))

In [50]:
ideas['abstract']=ideas['Title']+' '+ideas['Description']+' '+ideas['Importance']+' '+ideas['Keywords']+' '+ideas['Stakeholders']+' '+ideas['Methdologies']+" "+ ideas['Program']

In [67]:
ideas

,Unnamed: 0,Title,Description,Keywords,Categories,Stakeholders,Importance,Methdologies,Timeframes,Program,Miscellaneous,Author,History,abstract,tags,cleanAbstract,cleanTags,abstractWC,tagsWC,combinedAbsandTags
0,0,Visualization of Data About or From Tools in A...,Combining live sensor data and tools with AR d...,"connected tools, manufacturing, AR display, in...","Business, Technology, Use Cases, Industries","Operators of manufacturing, repair, maintenanc...",- AREA members with manufacturing facilities f...,An experimental environment using a range of c...,Medium,This research topic is broad in scope. It can ...,Two AREA members with financial support from b...,Christine Perey,2021-08-31,Visualization of Data About or From Tools in A...,"connected tools, manufacturing, AR display, in...",visualization data tool ar display combining l...,connected_tools manufacturing ar_display indus...,509,22,connected_tools manufacturing ar_display indus...
1,1,AR for Material Management and/or Supply Chain...,Many industries track supply chains and juggle...,"logistics data processing, materials managemen...","Business, Technology, Use Cases, End User and ...","operators of manufacturing, repair, employees ...",- AREA members with manufacturing facilities f...,An experimental environment using a range of m...,Medium,This research topic can be combined with studi...,Three non-commercial AREA members with financi...,Christine Perey,2021-08-31,AR for Material Management and/or Supply Chain...,"logistics data processing, materials managemen...",ar material management supply chain flow work ...,logistics_data_processing materials_management...,429,23,logistics_data_processing materials_management...
2,2,Dynamic Distribution of Sensing and Processing...,Deploying enterprise AR solutions via head-mou...,"distributed computing architecture, body-worn ...","Displays, Standards, Business, Technology","HMD manufacturers, AR display device designers...",- Employees in AREA member companies frequentl...,"Studies of wearable sensors, low-latency body ...",Long,This topic is at the intersection of multiple ...,Studies of optical and IMU sensor fusion for A...,"Peter Orban, Christine Perey",2021-08-31,Dynamic Distribution of Sensing and Processing...,"distributed computing architecture, body-worn ...",dynamic distribution sensing processing capaci...,distributed_computing_architecture body-worn_m...,411,20,distributed_computing_architecture body-worn_m...
3,3,Wearable vs Handheld vs Projection: Methods fo...,"Tablets, mobile phones, and other handheld dev...","efficiency, handheld, projection AR, wearable ...","Displays, Business","Operations leaders, financial management, OEM ...",- Wearable AR displays provide frontline worke...,The research will contribute to development of...,Near,This research can be combined with or extended...,The AREA RoI calculator is a starting point fo...,"Peter Orban, Christine Perey",2021-08-31,Wearable vs Handheld vs Projection: Methods fo...,"efficiency, handheld, projection AR, wearable ...",wearable v handheld v projection method quanti...,efficiency handheld projection_ar wearable_dis...,350,10,efficiency handheld projection_ar wearable_dis...
4,4,Impact of AR Delivery on People Living in Mult...,"The Multidimensional Poverty Index (MPI) ""meas...","United Nations, Sustainable Development Goals ...","Business, End User and User Experience",Government officials and policymakers in World...,- Because AR is a unique delivery format that ...,The proposed research would need to decide upo...,Near,This study could link closely with existing re...,"The Multidimensional Poverty Index (MPI) ""meas...",Jennifer Rogers,2021-08-31,Impact of AR Delivery on People Living in Mult...,"United Nations, Sustainable Development Goals ...",impact ar delivery people living multidimensio...,united_nations sustainable_development_goals_(...,440,12,united_nations sustainable_development_goals_(...
5,5,Passthrough vs. See-through: Impacts of Displ

In [52]:
ideas['tags']=ideas['Keywords']

In [53]:
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
#new_words = ["br", "use", "award","project","research","using","new","based","nsf","develop","application"] 
#stop_words = stop_words.union(new_words)

corpus = []
#df.drop(columns=['cleanAbstract'])

for i in range(0, len(ideas)):
    #Remove punctuations
    text = re.sub('[^a-zA-Z0-9]', ' ', ideas['abstract'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    #text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
    
ideas['cleanAbstract'] = corpus

In [54]:
idea_terms = []
for i in range(0,len(ideas)):
    text = ideas['tags'][i]
    text=text.lower()
    text = re.sub(', ',';', text)
    text = re.sub(' ','_',text)
    text = re.sub(';',' ',text)
    idea_terms.append(text)
ideas['cleanTags'] = idea_terms
    

In [55]:
word_count = []
for i in range(0,len(ideas)):
    words_ = ideas['cleanAbstract'][i].split(' ')
    word_count.append(len(words_))
ideas['abstractWC']=word_count

In [56]:
word_count = []
for i in range(0,len(ideas)):
    words_ = ideas['cleanTags'][i].split(' ')
    word_count.append(len(words_))
ideas['tagsWC']=word_count

In [57]:
ideas['combinedAbsandTags']=ideas['cleanTags']+ideas['cleanAbstract']

In [58]:
midlvl=[]

for i in range(0,len(dataset)):
    set_string = []
    terms = dataset["midlvl"][i].split(";")
    for j in range(0, len(terms)):
        set_string.append(terms[j])
    midlvl.append(set_string)


In [59]:
lowlvl=[]
for i in range(0,len(dataset)):
    set_string = []
    terms = str(dataset["lowlvl"][i]).split(";")
    for j in range(0, len(terms)):
        set_string.append(terms[j])
    lowlvl.append(set_string)

In [60]:
hlvl=[]
for i in range(0,len(dataset)):
    set_string = []
    terms = dataset["HighLevel"][i]
    for j in range(0, len(terms)):
        set_string.append(terms[j])
    hlvl.append(set_string)

In [61]:
document = lowlvl

# Get all of the unique entries you have
varnames_low = tuple(sorted(set(itertools.chain(*document))))

# Get a list of all of the combinations you have
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded)


# Create the table
table_low = np.zeros((len(varnames_low),len(varnames_low)), dtype=int)

for i, v1 in enumerate(varnames_low):
    for j, v2 in enumerate(varnames_low[i:]):        
        j = j + i 
        table_low[i, j] = c[v1, v2]
        table_low[j, i] = c[v1, v2]

# Display the output
#for row in table:
#    print(row)

In [62]:
document = midlvl

# Get all of the unique entries you have
varnames_mid = tuple(sorted(set(itertools.chain(*document))))

# Get a list of all of the combinations you have
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded)


# Create the table
table_mid = np.zeros((len(varnames_mid),len(varnames_mid)), dtype=int)

for i, v1 in enumerate(varnames_mid):
    for j, v2 in enumerate(varnames_mid[i:]):        
        j = j + i 
        table_mid[i, j] = c[v1, v2]
        table_mid[j, i] = c[v1, v2]

In [63]:
document = hlvl

# Get all of the unique entries you have
varnames_high = tuple(sorted(set(itertools.chain(*document))))

# Get a list of all of the combinations you have
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded)


# Create the table
table_high = np.zeros((len(varnames_high),len(varnames_high)), dtype=int)

for i, v1 in enumerate(varnames_high):
    for j, v2 in enumerate(varnames_high[i:]):        
        j = j + i 
        table_high[i, j] = c[v1, v2]
        table_high[j, i] = c[v1, v2]

In [71]:
mids = pandas.DataFrame()
mids['names'] = varnames_mid
mids['freq'] = np.zeros(len(varnames_mid))
mids['freq_2017']=np.zeros(len(varnames_mid))
mids['freq_2018']=np.zeros(len(varnames_mid))
mids['freq_2019']=np.zeros(len(varnames_mid))
mids['freq_2020']=np.zeros(len(varnames_mid))
mids['freq_2021']=np.zeros(len(varnames_mid))

freq_count = []
freq_count_2017 = []
freq_count_2018 = []
freq_count_2019 = []
freq_count_2020 = []
freq_count_2021 = []
for i in range(0,len(varnames_mid)):
    freq = 0
    mid_test = varnames_mid[i]
    for j in range(0,len(dataset)):
        test = dataset["midlvl"][j].split(";")
        year = dataset["Publication year"][j]
        freq_year=0
        for k in range(0,len(test)):
            if mid_test==test[k]:
                freq=freq+1
                freq_year=freq_year+1
        if dataset["Publication year"][j]==2017:
            mids['freq_2017'][i] = mids['freq_2017'][i] +freq_year
        if dataset["Publication year"][j]==2018:
            mids['freq_2018'][i] = mids['freq_2018'][i] +freq_year
        if dataset["Publication year"][j]==2019:
            mids['freq_2019'][i] = mids['freq_2019'][i] +freq_year
        if dataset["Publication year"][j]==2020:
            mids['freq_2020'][i] = mids['freq_2020'][i] +freq_year
        if dataset["Publication year"][j]==2021:
            mids['freq_2021'][i] = mids['freq_2021'][i] +freq_year            
    mids['freq'][i]=freq


mids.to_csv('AllMidLvlTerms.csv')

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is try

In [72]:
lows = pandas.DataFrame()
freq_count = []
for i in range(0,len(varnames_low)):
    freq=0
    low_test = varnames_low[i]
    for j in range(0,len(dataset)):
        test = dataset["lowlvl"][j].split(";")
        for k in range(0,len(test)):
            if low_test==test[k]:
                freq=freq+1
    freq_count.append(freq)

lows['names'] = varnames_low
lows['freq']= freq_count

lows.to_csv('AllLowLvlTerms.csv')
    

In [73]:
new_alias=pandas.DataFrame()
new_alias['LowLevel']=lows['names']
new_alias['MidLevel']=lows['names']
new_alias['HighLevel']=lows['names']
new_alias["freq_low"] =lows['freq']
new_alias["freq_mid"] =lows['freq']
new_alias["freq_high"]=lows['freq']
new_alias["freq_low"] =lows['freq']
new_alias["freq_mid"] =lows['freq']
new_alias["freq_high"]=lows['freq']

In [74]:
mids_list=[]
highs_list=[]
new_alias=pandas.DataFrame()
new_alias['LowLevel']=lows['names']
new_alias['MidLevel']=lows['names']
new_alias['HighLevel']=lows['names']
new_alias["freq_low"] =lows['freq']
new_alias["freq_mid"] =lows['freq']
new_alias["freq_high"]=lows['freq']
for i in range(len(new_alias)):
    term1 = new_alias['LowLevel'][i]
    for j in range(len(alias)):
        term2 = alias['LowLevel'][j]
        if term1==term2:
            new_alias['MidLevel'][i] = alias['MidLevel'][j]
            new_alias['HighLevel'][i]= alias['HighLevel'][j]
#new_alias['MidLevel'] = mids
#new_alias['HighLevel'] = highs

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [75]:
for i in range(len(new_alias)):
    term1 = new_alias['LowLevel'][i]
    for j in range(len(alias)):
        term2 = alias['LowLevel'][j]
        if term1==term2:
            new_alias['MidLevel'][i] = alias['MidLevel'][j]
            new_alias['HighLevel'][i]= alias['HighLevel'][j]

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [76]:
industry_terms = []
for i in range(0, len(new_alias)):
    hl = new_alias["HighLevel"][i]
    if hl=="industries":
        industry_terms.append(new_alias["MidLevel"][i])
industry_terms=list(set(industry_terms))

In [77]:
industry_df = pandas.DataFrame()
industry_df['names']=industry_terms
ind_freq = []
for i in range(0, len(industry_df)):
    freq = 0
    term1 = industry_df['names'][i]
    for j in range(0,len(dataset)):
        mid_terms = dataset['MidLevel'][j]
        for k in range(0, len(mid_terms)):
            term2=mid_terms[k]
            if term1==term2:
                freq=freq+1
    ind_freq.append(freq)            
industry_df['freq']=ind_freq    

In [78]:
ilvl = []
for i in range(0,len(dataset)):
    set_string = []
    terms = dataset["MidLevel"][i]
    for j in range(0, len(terms)):
        for k in range(0, len(industry_df)):
            tst = industry_df['names'][k]
            if terms[j]==tst:
                set_string.append(str(terms[j]))
    ilvl.append(set_string)

In [79]:
document = ilvl

# Get all of the unique entries you have
varnames_i = tuple(sorted(set(itertools.chain(*document))))

# Get a list of all of the combinations you have
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded)


# Create the table
table_i = np.zeros((len(varnames_i),len(varnames_i)), dtype=int)

for i, v1 in enumerate(varnames_i):
    for j, v2 in enumerate(varnames_i[i:]):        
        j = j + i 
        table_i[i, j] = c[v1, v2]
        table_i[j, i] = c[v1, v2]

In [80]:
highs = pandas.DataFrame()
freqs=[]
high_names = ['business','displays','end users and user experience','industries','other','standards','technology','use cases']
highs['names']=high_names
for j in range(0,len(highs)):
    freq=0
    term1=highs['names'][j]
    for i in range(0,len(dataset)):
        term_set=dataset['HighLevel'][i]
        for k in range(0, len(term_set)):
            term2=term_set[k]
            if term1==term2:
                freq=freq+1
    freqs.append(freq)
highs['freq']=freqs

In [ ]:

for i in range(0,len(high_names)):
    high_term = high_names[i]
    mid_terms = []
    low_terms = []
    for j in range(0,len(new_alias)):
        mid_term = new_alias['MidLevel'][j]
        low_term = new_alias['LowLevel'][j]
        if new_alias["HighLevel"][j]==high_term:
            mid_terms.append(mid_term)
            low_terms.append(low_term)
    mid_terms=list(set(mid_terms))
    low_terms=list(set(low_terms))
    print(high_term)
    print("number of groups: "+str(len(mid_terms)))
    print("number of machine-generated terms: "+str(len(low_terms)))

In [ ]:
for i in range(0,len(new_alias)):
    term1=new_alias["LowLevel"][i]
    for j in range(0, len(lows)):
        term2 = lows["names"][j]
        if term1==term2:
            new_alias["freq_low"][i]=lows['freq'][j]
            

            

In [ ]:

for i in range(len(new_alias)):
    for j in range(len(mids)):
        term1=new_alias['MidLevel'][i]
        term2=mids['names'][j]
        if term1==term2:
            new_alias["freq_mid"][i]=mids['freq'][j]

In [ ]:
for i in range(len(new_alias)):
    for j in range(len(highs)):
        term1=new_alias['HighLevel'][i]
        term2=highs['names'][j]
        if term1==term2:
            new_alias["freq_high"][i]=highs['freq'][j]

In [ ]:
new_alias_tree=new_alias
for i in range(0,len(new_alias_tree)):
    new_alias_tree['LowLevel'][i]= str(new_alias_tree['LowLevel'][i])+'__'
    new_alias_tree['MidLevel'][i]= str(new_alias_tree['MidLevel'][i])+'_'

In [ ]:
sankey_full = pandas.DataFrame(columns=list('ABC'))
for i in range(0, len(mids)):
    for j in range(0, len(new_alias_tree)):
        mid_term = mids['names'][i]+'_'
        mid_alias = new_alias_tree['MidLevel'][j]
        if mid_term==mid_alias:
            df2 = pandas.DataFrame([[new_alias_tree['HighLevel'][j], new_alias_tree['MidLevel'][j], new_alias_tree['freq_mid'][j]]], columns=list('ABC'))
            sankey_full=sankey_full.append(df2,ignore_index=True)
            break
for i in range(0, len(lows)):
    for j in range(0, len(new_alias_tree)):
        low_term = lows['names'][i]+'__'
        low_alias = new_alias_tree['LowLevel'][j]
        if low_term==low_alias:
            df2 = pandas.DataFrame([[new_alias_tree['MidLevel'][j], new_alias_tree['LowLevel'][j], new_alias_tree['freq_low'][j]]], columns=list('ABC'))
            sankey_full=sankey_full.append(df2,ignore_index=True)
            break

In [ ]:
sankey_full["D"] = np.nan

In [ ]:
for i in range(0,len(sankey_full)):
    for j in range(0,len(varnames_high)):
        if sankey_full['A'][i]==varnames_high[j]:
            sankey_full['D'][i]=j

In [ ]:
for i in range(0,len(sankey_full)):
    for j in range(0,len(new_alias_tree)):
        if sankey_full['A'][i]==new_alias_tree['LowLevel'][j]:
            for k in range(0,len(varnames_high)):
                if new_alias_tree['HighLevel'][j]==varnames_high[k]:
                    sankey_full['D'][i]=k
        if sankey_full['A'][i]==new_alias_tree['MidLevel'][j]:
            for k in range(0,len(varnames_high)):
                if new_alias_tree['HighLevel'][j]==varnames_high[k]:
                    sankey_full['D'][i]=k
        

In [ ]:
sankey_full.to_csv("sankey_full.csv")

In [ ]:
f = open("tree-high.txt", "w")
for i in range (0, len(sankey)):
    f.write("[ '"+str(sankey['B'][i])+"', '"+str(sankey['A'][i])+"', "+str(sankey['C'][i])+", "+str(sankey['D'][i])+ "],\n")
f.close()

In [ ]:
for i in range(0, len(sankey_full)):
    sankey_full['D'][i]=int(sankey_full['D'][i])

In [ ]:
f = open("tree-full.txt", "w")
for i in range (0, len(sankey_full)):
    f.write("[ '"+str(sankey_full['B'][i])+"', '"+str(sankey_full['A'][i])+"', "+str(sankey_full['C'][i])+", "+str(sankey_full['D'][i])+ "],\n")
f.close()

In [ ]:
f = open("sankey-full.txt", "w")
for i in range (0, len(sankey_full)):
    f.write("[ '"+str(sankey_full['A'][i])+"', '"+str(sankey_full['B'][i])+"', "+str(sankey_full['C'][i])+", "+str(sankey_full['D'][i])+ "],\n")
f.close()

In [ ]:
np.savetxt("table-low.csv", table_low, delimiter=",")

In [ ]:
new_alias_tree = alias
for i in range(0,len(new_alias_tree)):
    low_hold = new_alias_tree["LowLevel"][i]
    mid_hold = new_alias_tree["MidLevel"][i]
    high_hold = new_alias_tree["HighLevel"][i]
    if low_hold==mid_hold:
        new_alias_tree["LowLevel"][i]=low_hold+"__"
    if mid_hold==high_hold:
        new_alias_tree["MidLevel"][i]=mid_hold+"_"    
    

In [ ]:
sankey_low.to_csv("sankey-low.csv")

In [ ]:
sankey_low["D"] = np.nan

In [ ]:
sankey_full

In [ ]:
f = open("tree-low.txt", "w")
for i in range (0, len(sankey_low)):
    f.write("[ '"+str(sankey_low['B'][i])+"', '"+str(sankey_low['A'][i])+"', "+str(sankey_low['C'][i])+", "+str(sankey_low['D'][i])+ "],\n")
f.close()

In [ ]:
high_terms = []
for i in range(0,len(dataset)):
    high_ = dataset['HighLevel'][i]
    string_= ''
    if len(high_) == 0:
        string_ = 'other'
    else: 
        for j in range(0, len(high_)):
            if j==0:
                string_ = high_[j]
            if j!=0:
                string_ = string_ +';'+high_[j]
    high_terms.append(string_)
    
dataset['highlvl']=high_terms
        

In [ ]:
dataset.to_csv("test.csv", encoding='utf-8-sig')


In [ ]:
highlvl=[]
for i in range(0,len(dataset)):
    term_set = dataset['LowLevel'][i]
    highs_=[]
    for j in range(0,len(term_set)):
        low_term1 = term_set[j]
        for k in range(0, len(new_alias)):
            low_term2 = new_alias['LowLevel'][k]
            if low_term1==low_term2:
                highs_.append(new_alias["HighLevel"][k])
    highs_=list(set(highs_))
    highlvl.append(highs_)
            

In [ ]:
high_pair = []
for i in range(0,len(varnames_high)):
    tst = varnames_high[i]
    for j in range(0,len(varnames_high)):
        tst2 = varnames_high[j]
        pair = [tst,tst2]
        high_pair.append(pair)
high1 = []
high2 = []
for i in range(0, len(high_pair)):
    high1.append(high_pair[i][0])
    high2.append(high_pair[i][1])

co_value_high =[]
for i in range(0,len(table_high)):
    for j in range(0, len(table_high)):
        co_value_high.append(table_high[i,j])

In [ ]:
ind_pair = []
for i in range(0,len(varnames_i)):
    tst = varnames_i[i]
    for j in range(0,len(varnames_i)):
        tst2 = varnames_i[j]
        pair = [tst,tst2]
        ind_pair.append(pair)
ind1 = []
ind2 = []
for i in range(0, len(ind_pair)):
    ind1.append(ind_pair[i][0])
    ind2.append(ind_pair[i][1])

co_value_ind =[]
for i in range(0,len(table_i)):
    for j in range(0, len(table_i)):
        co_value_ind.append(table_i[i,j])

In [ ]:
mid_pair = []
for i in range(0,len(varnames_mid)):
    tst = varnames_mid[i]
    for j in range(0,len(varnames_mid)):
        tst2 = varnames_mid[j]
        pair = [tst,tst2]
        mid_pair.append(pair)

In [ ]:
mid1 = []
mid2 = []
for i in range(0, len(mid_pair)):
    mid1.append(mid_pair[i][0])
    mid2.append(mid_pair[i][1])

In [ ]:
co_value =[]
for i in range(0,len(table_mid)):
    for j in range(0, len(table_mid)):
        co_value.append(table_mid[i,j])

In [ ]:
with open('low_terms.txt', 'w') as f:
    for item in varnames_low:
        f.write("%s\n" % item)

In [ ]:
new_alias.to_csv("new_alias.csv")

In [ ]:
low_pair = []
for i in range(0,len(varnames_low)):
    tst = varnames_low[i]
    for j in range(0,len(varnames_low)):
        tst2 = varnames_low[j]
        pair = [tst,tst2]
        low_pair.append(pair)

In [ ]:
low1 = []
low2 = []
for i in range(0, len(low_pair)):
    low1.append(low_pair[i][0])
    low2.append(low_pair[i][1])

In [ ]:
co_value_low =[]
for i in range(0,len(table_low)):
    for j in range(0, len(table_low)):
        co_value_low.append(table_low[i,j])

In [ ]:
df_table_mid = pandas.DataFrame(table_mid)
df_viz = pandas.DataFrame()
df_viz["Mid1"]=mid1
df_viz["Mid2"]=mid2
df_viz["value"]=co_value

df_viz_low= pandas.DataFrame()
df_viz_low["Low1"]=low1
df_viz_low["Low2"]=low2
df_viz_low["value"]=co_value_low

df_viz_high= pandas.DataFrame()
df_viz_high["High1"]=high1
df_viz_high["High2"]=high2
df_viz_high["value"]=co_value_high

df_viz_ind= pandas.DataFrame()
df_viz_ind["Ind1"]=ind1
df_viz_ind["Ind2"]=ind2
df_viz_ind["value"]=co_value_ind



In [ ]:
counter = 0
for i in range(0, len(dataset)):
    if dataset['Relevant'][i]==1:
        counter=counter+1
        dataset['Title'][i]="*"+str(dataset['Title'][i])
print(counter)
        

In [ ]:
inds = pandas.DataFrame()
inds['names'] = list(set(df_viz_ind["Ind1"]))
freq = []
freq_17 = []
freq_18 = []
freq_19 = []
freq_20 = []
freq_21 = []
for i in range(0, len(inds)):
    term1 = inds["names"][i]
    for j in range(0, len(mids)):
        term2 = mids["names"][j]
        if term1==term2:
            freq.append(mids["freq"][j])
            freq_17.append(mids["freq_2017"][j])
            freq_18.append(mids["freq_2018"][j])
            freq_19.append(mids["freq_2019"][j])
            freq_20.append(mids["freq_2020"][j])
            freq_21.append(mids["freq_2021"][j])

inds['freq'] = freq
inds['freq_2017'] = freq_17
inds['freq_2018'] = freq_18
inds['freq_2019'] = freq_19
inds['freq_2020'] = freq_20
inds['freq_2021'] = freq_21


In [ ]:
inds.to_csv("inds.csv")
mids.to_csv("mids.csv")
lows.to_csv("lows.csv")
highs.to_csv("highs.csv")
dataset.to_csv("data-Mar11.csv")

In [ ]:
proc = subprocess.Popen('curl -LH "Accept: application/rdf+xml;q=0.5, application/vnd.citationstyles.csl+json;q=1.0" '+str(dataset["DOI"][2]),stdout=subprocess.PIPE)
output = proc.stdout.read()
new=output.decode('ascii')
print(new)


In [ ]:
y = json.loads(new) 

In [ ]:
dataset['newNames']=""


In [ ]:
apa_citation = []
for i in range(0, len(dataset)):
    proc = subprocess.Popen('curl -LH "Accept: text/x-bibliography; style=apa" '+str(dataset['DOI'][i]), stdout=subprocess.PIPE)
    output = proc.stdout.read()
    apa_citation.append(output)
    print(i)
dataset['Citation']=apa_citation

In [ ]:
author_names = []
for j in range(1592,len(dataset)):
    proc = subprocess.Popen('curl -LH "Accept: application/rdf+xml;q=0.5, application/vnd.citationstyles.csl+json;q=1.0" '+str(dataset["DOI"][j]),stdout=subprocess.PIPE)
    output = proc.stdout.read()
    new=output.decode('ascii')
    #print(new)
    try:
        y = json.loads(new)    
        author_string = ''
        for i in range (0,len(y['author'])):
            if i < len(y['author'])-1:
                author_string = author_string + y['author'][i]['given'].encode('latin1').decode('unicode-escape') +' '+y['author'][i]['family'].encode('latin1').decode('unicode-escape')+', '
            else:
                if len(y['author'])==1:
                    author_string = y['author'][i]['given'].encode('latin1').decode('unicode-escape')+' '+y['author'][i]['family'].encode('latin1').decode('unicode-escape')
                else:
                    author_string = author_string + 'and ' + y['author'][i]['given'].encode('latin1').decode('unicode-escape')+' '+y['author'][i]['family'].encode('latin1').decode('unicode-escape')
        author_names.append(author_string)
        dataset['newNames'][j]=author_string
        print(author_string)
        print(j)
    except:
        print('skipped!')
        print(j)
        




In [ ]:
dataset.to_csv("data_Feb26.csv", encoding='utf-8-sig')

In [ ]:
dataset = pandas.read_csv('test.csv',encoding = "ISO-8859-1")

In [ ]:
author_string = ''
for i in range (0,len(y['author'])):
    if i < len(y['author'])-1:
        author_string = author_string + y['author'][i]['given'].encode('latin1').decode('unicode-escape') +' '+y['author'][i]['family'].encode('latin1').decode('unicode-escape')+', '
    else:
        author_string = author_string + 'and ' + y['author'][i]['given'].encode('latin1').decode('unicode-escape')+' '+y['author'][i]['family'].encode('latin1').decode('unicode-escape')
print(author_string)

In [ ]:
df = dataset
for i in range (0,len(df)):
    string = df['newNames'][i]
    string = re.sub('(*','',string)
    df['newNames'][i] = string

In [64]:
business_names = ['business performance metrics','business planning and management','human resources','integration',
                  'logistics','policy','sales and marketing']
display_names = ['device energy management','display technology','optics','wearables']
endusers_names = ['human factors','human-computer interaction','presence','users']
industries_names = ['automotive','aviation and aerospace','chemical','construction','consumer products','cultural heritage',
                    'education','emergency response','farming and natural science','government','industrial equipment',
                    'liberal arts','manufacturing','marine','medical','metals and mining','oil and gas','power and energy',
                    'telecommunication','transportation','utilities']
usecases_names = ['collaboration','inspection, safety and quality','navigation','simulation','smart cities','training']
technology_names = ['artificial intelligence','audio','computer vision','data','developers','engineering',
                    'geospatial','graphics','input','internet of things','networks','robotics','security',
                    'semiconductors','sensors','video','web services']
standards_names = ['standards']
other_names = ['other']

In [65]:
df_industries= pandas.DataFrame() 
df_business = pandas.DataFrame()
df_technology = pandas.DataFrame()
df_usecases = pandas.DataFrame()
df_displays = pandas.DataFrame()
df_endusers = pandas.DataFrame()
df_standards = pandas.DataFrame()
df_other = pandas.DataFrame()






In [81]:
df_industries['names'] = varnames_i
df_business['names'] = business_names
df_technology['names'] = technology_names
df_usecases['names'] = usecases_names
df_displays['names'] = display_names
df_endusers['names'] = endusers_names
df_standards['names'] = standards_names
df_other['names'] = other_names

In [82]:
ideas['hlvl']=ideas['Categories']

In [ ]:
instance = df_other
freq = []
for i in range(0, len(instance)):    
    for j in range(0, len(mids)):
        middle_term = mids['names'][j]
        other_term = instance['names'][i]
        if middle_term == other_term:
            freq.append(mids['freq'][j])
            
instance['freq'] = freq
df_other = instance
        

In [83]:
new_terms = []
for i in range(0,len(ideas)):
    ideas['hlvl'][i] = ideas['hlvl'][i].lower()
    ideas['hlvl'][i] = re.sub(", ",";",ideas['hlvl'][i])
for i in range(0,len(ideas)):
    terms_h = ideas['hlvl'][i].split(';')
    for j in range(0,len(terms_h)):
        if terms_h[j]=='display devices':
            terms_h[j] = 'displays'
        if terms_h[j]=='end users':
            terms_h[j] = 'end users and user experience'
    new_terms.append(terms_h)
ideas['newTags']=new_terms

C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\wzb\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
colors = ["#083A49", "#0B4352","#0f4c5b","#125563","#165e6c",
          "#196775","#1C707E","#207986","#23828F","#278B98","#2a94a1","#2e9da9","#31A6B2"]
qualitative_colors = sns.color_palette(colors)
sns.palplot(qualitative_colors)

import matplotlib.colors as clr
cmap = clr.LinearSegmentedColormap.from_list('area_colors', ['#31A6B2','#083A49'], N=256)
#customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 10}

plt.rc('font', **font)

In [ ]:

from matplotlib.font_manager import FontProperties

fontP = FontProperties()
fontP.set_size('small')

fig = plt.figure()
ax = plt.axes()

colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', 
          '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', 
          '#9a6324', '#fffac8', '#800000', 
          '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']
    
qualitative_colors = sns.color_palette(colors)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)

inds = inds.sort_values(by='freq_2017', ascending=False)

sum_2020_2021 = sum(inds["freq_2020"])+sum(inds["freq_2021"])

for i in range(0, len(inds)):
    x_values = [2017, 2018, 2019, 2020]
    y_values = [inds["freq_2017"][i]/sum(inds["freq_2017"]), inds["freq_2018"][i]/sum(inds["freq_2018"]), 
                inds["freq_2019"][i]/sum(inds["freq_2019"]), 
                (inds["freq_2020"][i]+inds["freq_2021"][i])/sum_2020_2021]
    plt.plot(x_values, y_values, label=inds['names'][i])

plt.xticks([2017,2018,2019,2020])
ax.legend(loc='upper left',bbox_to_anchor=(1.05, 1),prop=fontP)
plt.title('Relative contribution of industry term per year')
plt.show()







In [ ]:
df_viz = df_viz.sort_values(by='value', ascending=False)

a4_dims = (2.5, 2.5)

fig, ax = plt.subplots(figsize=a4_dims)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)
sns.scatterplot(ax=ax,
    data=df_viz[:50], x="Mid1", y="Mid2", hue="value", size="value",
    sizes=(50, 250),legend=False,palette=cmap
)
plt.xticks(rotation=90)
ax.set(xlabel=None)
ax.set(ylabel=None)


In [ ]:
df_viz_ind = df_viz_ind.sort_values(by='value', ascending=False)

a4_dims = (5, 5)

fig, ax = plt.subplots(figsize=a4_dims)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)
sns.scatterplot(ax=ax,
    data=df_viz_ind, x="Ind1", y="Ind2", hue="value", size="value",
    sizes=(1, 1000),legend=False,palette=cmap
)
plt.xticks(rotation=90)
ax.set(xlabel=None)
ax.set(ylabel=None)

In [ ]:
df_viz_low = df_viz_low.sort_values(by='value', ascending=False)

a4_dims = (5, 5)

fig, ax = plt.subplots(figsize=a4_dims)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)
sns.scatterplot(ax=ax,
    data=df_viz_low[:50], x="Low1", y="Low2", hue="value", size="value",
    sizes=(100, 1000),legend=False,palette=cmap
)
plt.xticks(rotation=90)
ax.set(xlabel=None)
ax.set(ylabel=None)


In [ ]:
df_viz_high = df_viz_high.sort_values(by='value', ascending=False)

a4_dims = (3, 3)

fig, ax = plt.subplots(figsize=a4_dims)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)
sns.scatterplot(ax=ax,
    data=df_viz_high[:40], x="High1", y="High2", hue="value", size="value",
    sizes=(100, 1000),legend=False,palette=cmap
)
plt.xticks(rotation=90)
ax.set(xlabel=None)
ax.set(ylabel=None)

In [ ]:
p=df_viz.sort_values(by='value', ascending=False)

In [ ]:
# Create an array with the colors you want to use
#colors = ["#083A49", "#31A6B2"]
# Set your custom color palette
 
a4_dims = (5, 5)

colors = ["#083A49", "#0B4352","#0f4c5b","#125563","#165e6c",
          "#196775","#1C707E","#207986","#23828F","#278B98","#2a94a1","#2e9da9","#31A6B2"]
fig, ax = plt.subplots(figsize=a4_dims)
plt.gcf().subplots_adjust(left=0.450, bottom=0.45)
customPalette = sns.set_palette(sns.color_palette(colors),n_colors=13)
#sns.color_palette(customPalete)
#sns.color_palette("mako")
ax = sns.scatterplot(data=p[:50], x="Mid1", y="Mid2", size="value",hue='value',
    sizes=(1, 1000),legend=False,palette=cmap
)
plt.xticks(rotation=90)
ax.set(xlabel=None)
ax.set(ylabel=None)
plt.savefig("mid-correlation-top50-hi-res.png",dpi=100)


In [ ]:
df_table_mid = pandas.DataFrame(table_mid)
sns.clustermap(df_table_mid,xticklabels=varnames_mid, yticklabels=varnames_mid)
#sns.heatmap(table, xlabel=midlvl_names)

In [ ]:
df_table_low = pandas.DataFrame(table_low)
sns.clustermap(df_table_low,xticklabels=varnames_low, yticklabels=varnames_low)

In [ ]:
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

    
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        
plt.rcdefaults()
fig,ax = plt.subplots()
# fig, (ax1,ax2,ax3,ax4,ax5,ax6) = plt.subplots(1,6)
right_side = ax.spines["right"]
right_side.set_visible(False)

# right_side = ax2.spines["right"]
# right_side.set_visible(False)

# right_side = ax3.spines["right"]
# right_side.set_visible(False)

# right_side = ax4.spines["right"]
# right_side.set_visible(False)

# right_side = ax5.spines["right"]
# right_side.set_visible(False)

# right_side = ax6.spines["right"]
# right_side.set_visible(False)
#ax.plot([1,6])


set_size(5,2)
plt.gcf().subplots_adjust(left=0.60)

# Example data
mids=mids.sort_values(by=['freq'],ascending=False)
highs=highs.sort_values(by=['freq'],ascending=False)
lows=lows.sort_values(by=['freq'],ascending=False)
industry_df=industry_df.sort_values(by=['freq'],ascending=False)


df_industries.sort_values(by=['freq'],ascending=False)

df_business.sort_values(by=['freq'],ascending=False)
df_technology.sort_values(by=['freq'],ascending=False)
df_usecases.sort_values(by=['freq'],ascending=False)
df_displays.sort_values(by=['freq'],ascending=False)
df_endusers.sort_values(by=['freq'],ascending=False)
df_standards.sort_values(by=['freq'],ascending=False)
df_other.sort_values(by=['freq'],ascending=False)

df_instance = highs.sort_values(by=['freq'],ascending=False)


mid_terms = mids["names"]
mid_freq = mids["freq"]
low_terms = lows["names"]
low_freq = lows["freq"]
mid_freq_2017 = mids["freq_2017"]
mid_freq_2018 = mids["freq_2018"]
mid_freq_2019 = mids["freq_2019"]
mid_freq_2020 = mids["freq_2020"]
mid_freq_2021 = mids["freq_2021"]

y_pos = np.arange(len(df_instance))



rects = ax.barh(y_pos, df_instance["freq"],color=qualitative_colors)
ax.set_yticks(y_pos)
ax.set_yticklabels(df_instance["names"])
ax.invert_yaxis()  # labels read top-to-bottom
plt.xlim([0, max(df_instance["freq"])])
ax.set_xlabel('Count')
ax.set_title('Frequencies')
for i, v in enumerate(df_instance["freq"]):
    ax.text(v, i + 0.25, str(int(v)), color='black', fontweight='normal')
#fig.savefig("mids.pdf")
#plt.show()


# rects = ax2.barh(y_pos, mid_freq_2017)
# ax2.set_yticks(y_pos)
# #ax2.set_yticklabels(mid_terms)
# ax2.invert_yaxis()  # labels read top-to-bottom
# ax2.set_xlabel('Count')
# ax2.set_title('2017')
# for i, v in enumerate(mid_freq_2017):
#     ax2.text(v + 10, i + 0.25, str(int(v)), color='black', fontweight='normal')
# #fig.savefig("mids_2017.pdf")


# rects = ax3.barh(y_pos, mid_freq_2018)
# ax3.set_yticks(y_pos)
# #ax3.set_yticklabels(mid_terms)
# ax3.invert_yaxis()  # labels read top-to-bottom
# ax3.set_xlabel('Count')
# ax3.set_title('2018')
# for i, v in enumerate(mid_freq_2018):
#     ax3.text(v + 10, i + 0.25, str(int(v)), color='black', fontweight='normal')
# #fig.savefig("mids_2018.pdf")

# rects = ax4.barh(y_pos, mid_freq_2019)
# ax4.set_yticks(y_pos)
# #ax4.set_yticklabels(mid_terms)
# ax4.invert_yaxis()  # labels read top-to-bottom
# ax4.set_xlabel('Count')
# ax4.set_title('2019')
# for i, v in enumerate(mid_freq_2019):
#     ax4.text(v + 10, i + 0.25, str(int(v)), color='black', fontweight='normal')
# #fig.savefig("mids_2019.pdf")

# rects = ax5.barh(y_pos, mid_freq_2020)
# ax5.set_yticks(y_pos)
# #ax5.set_yticklabels(mid_terms)
# ax5.invert_yaxis()  # labels read top-to-bottom
# ax5.set_xlabel('Count')
# ax5.set_title('2020')
# for i, v in enumerate(mid_freq_2020):
#     ax5.text(v + 10, i + 0.25, str(int(v)), color='black', fontweight='normal')
# #fig.savefig("mids_2020.pdf")

# rects = ax6.barh(y_pos, mid_freq_2021)
# ax6.set_yticks(y_pos)
# ax.set_yticklabels("")
# ax6.invert_yaxis()  # labels read top-to-bottom
# ax6.set_xlabel('Count')
# ax6.set_title('2021')
# for i, v in enumerate(mid_freq_2021):
#     ax6.text(v + 10, i + 0.25, str(int(v)), color='black', fontweight='normal')
plt.savefig("standards.png",dpi=1000)
plt.show()



In [ ]:
lows=lows.sort_values(by=['freq'],ascending=False)


for j in range(0,40):
    page=j+1

    plt.rcdefaults()
    fig, ax = plt.subplots()

#ax.plot([1,3,2])

    set_size(5,10)
    plt.gcf().subplots_adjust(left=0.50)

    if page==1:
        low_terms = lows["names"][0:61]
        low_freq = lows["freq"][0:61]
    else:
        first_term = 62*j
        last_term = first_term+61
        low_terms = lows["names"][first_term:last_term]
        low_freq = lows["freq"][first_term:last_term]
    y_pos = np.arange(len(low_terms))



    rects = ax.barh(y_pos, low_freq)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(low_terms)
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Count')
    ax.set_title('Frequency of Low-Level Terms')
    for i, v in enumerate(low_freq):
        ax.text(v, i + 0.25, str(v), color='black', fontweight='normal')

#for index, value in enumerate(mid_freq):
    #plt.text(value, index, str(value))

#plt.figure(figsize=(1,100))
    fig.savefig("low_page"+str(page)+".pdf")


In [84]:
ideas['TagsPlusAbstract'] = ideas['cleanAbstract'] + ideas['cleanTags']

In [85]:
TFidf_scores = []
#int(TFidf_scores)
for i in range(0,len(ideas)):
    print(i)
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(ideas['TagsPlusAbstract'][i],dataset['IdeasTagsPlusAbstract'][j])
        row_scores.append(score)
        #rint(score)
        #TFidf_scores[i,j].astype(float)
        #Tfidf_scores[i,j] = score
    TFidf_scores.append(row_scores)
ideas['Sim Scores']=TFidf_scores
means = []
stdevs = []
maxes=[]
for i in range(0, len(ideas)):
    mean = np.mean(ideas['Sim Scores'][i])
    stdev = np.std(ideas['Sim Scores'][i])
    max_score = max(ideas['Sim Scores'][i])
    means.append(mean)
    stdevs.append(stdev)
    maxes.append(max_score)
ideas['Mean Sim']=means
ideas['Stdev Sim']=stdevs
ideas['Max Score']=maxes

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


In [ ]:
dataset['business']=np.nan
dataset['displays']=np.nan
dataset['end users and user experience']=np.nan
dataset['industries']=np.nan
dataset['technology']=np.nan
dataset['use cases']=np.nan
dataset['standards']=np.nan
for i in range(0, len(dataset)):
    for j in range(0, len(dataset['HighLevel'][i])):
        if dataset['HighLevel'][i][j] == 'business':
            dataset['business'][i] = 1
        if dataset['HighLevel'][i][j] == 'displays':
            dataset['displays'][i] = 1
        if dataset['HighLevel'][i][j] == 'end users and user experience':
            dataset['end users and user experience'][i] = 1
        if dataset['HighLevel'][i][j] == 'industries':
            dataset['industries'][i] = 1   
        if dataset['HighLevel'][i][j] == 'technology':
            dataset['technology'][i] = 1
        if dataset['HighLevel'][i][j] == 'use cases':
            dataset['use cases'][i] = 1            
        if dataset['HighLevel'][i][j] == 'standards':
            dataset['standards'][i] = 1           
df_tech = dataset.loc[(dataset["technology"] == 1), ["IdeasTagsPlusAbstract"]]
df_business = dataset.loc[(dataset["business"] == 1), ["IdeasTagsPlusAbstract"]]
df_displays = dataset.loc[(dataset["displays"] == 1), ["IdeasTagsPlusAbstract"]]
df_users = dataset.loc[(dataset["end users and user experience"] == 1), ["IdeasTagsPlusAbstract"]]
df_industries = dataset.loc[(dataset["industries"] == 1), ["IdeasTagsPlusAbstract"]]
df_usecases = dataset.loc[(dataset["use cases"] == 1), ["IdeasTagsPlusAbstract"]]
df_standards = dataset.loc[(dataset["standards"] == 1), ["IdeasTagsPlusAbstract"]]

In [ ]:
df0 = pandas.concat([df_tech,df_business,df_displays]).drop_duplicates().reset_index(drop=True)
df1 = pandas.concat([df_tech,df_displays]).drop_duplicates().reset_index(drop=True)
df2 = pandas.concat([df_tech,df_users,df_displays]).drop_duplicates().reset_index(drop=True)
df3 = pandas.concat([df_tech,df_industries,df_displays]).drop_duplicates().reset_index(drop=True)
df4 = pandas.concat([df_industries,df_business,df_users]).drop_duplicates().reset_index(drop=True)
df5 = pandas.concat([df_industries,df_tech,df_usecases]).drop_duplicates().reset_index(drop=True)
df6 = pandas.concat([df_standards,df_tech]).drop_duplicates().reset_index(drop=True)
df7 = pandas.concat([df_industries,df_tech,df_business]).drop_duplicates().reset_index(drop=True)
df8 = pandas.concat([df_tech,df_industries,df_business]).drop_duplicates().reset_index(drop=True)
slice_sizes = [
    len(df0),
    len(df1),
    len(df2),
    len(df3),
    len(df4),
    len(df5),
    len(df6),
    len(df7),
    len(df8)
]
#ideas['Slice Sizes'] = slice_sizes

In [ ]:
means_slice = []
stdevs_slice = []
maxes_slice=[]

In [ ]:
baseline = pandas.DataFrame()
baseline['Academic']=baselineAcad
baseline['Fiction']=baselineFict
##baselining scores
TFidf_scores = []
means_slice = []
stdevs_slice = []
maxes_slice=[]
for i in range(0, len(baseline)):
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(baseline['Academic'][i],dataset['TagsPlusAbstract'][j])
        row_scores.append(score)
    mean = np.mean(row_scores)
    stdev = np.std(row_scores)
    max_score = max(row_scores)
    means_slice.append(mean)
    stdevs_slice.append(stdev)
    maxes_slice.append(max_score)
    print(str(len(row_scores)) + ' ' + str(mean)+ ' ' + str(stdev)+ ' ' + str(max_score))
    TFidf_scores.append(row_scores)
baseline['Acad Sim Scores']=TFidf_scores
baseline['Acad Mean Sim']=means_slice
baseline['Acad Stdev Sim']=stdevs_slice
baseline['Acad Max Score']=maxes_slice

In [ ]:
TFidf_scores = []
means_slice = []
stdevs_slice = []
maxes_slice=[]
for i in range(0, len(baseline)):
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(baseline['Fiction'][i],dataset['TagsPlusAbstract'][j])
        row_scores.append(score)
    mean = np.mean(row_scores)
    stdev = np.std(row_scores)
    max_score = max(row_scores)
    means_slice.append(mean)
    stdevs_slice.append(stdev)
    maxes_slice.append(max_score)
    #print(str(len(row_scores)) + ' ' + str(mean)+ ' ' + str(stdev)+ ' ' + str(max_score))
    TFidf_scores.append(row_scores)
    print(i)
baseline['Fict Sim Scores']=TFidf_scores
baseline['Fict Mean Sim']=means_slice
baseline['Fict Stdev Sim']=stdevs_slice
baseline['Fict Max Score']=maxes_slice

In [ ]:
baseline.to_csv('basline.csv')

In [ ]:
ind_dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8]

In [ ]:
baseline = pandas.read_csv('basline.csv')

In [ ]:
TFidf_scores = []
means_slice = []
stdevs_slice = []
maxes_slice=[]
for i in range(0, len(ind_dfs)):
    row_scores =[]
    for j in range(0,len(ind_dfs[i])):
        score = cosine_sim(ideas['TagsPlusAbstract'][i],ind_dfs[i]['IdeasTagsPlusAbstract'][j])
        row_scores.append(score)
    mean = np.mean(row_scores)
    stdev = np.std(row_scores)
    max_score = max(row_scores)
    means_slice.append(mean)
    stdevs_slice.append(stdev)
    maxes_slice.append(max_score)
    print(str(len(row_scores)) + ' ' + str(mean)+ ' ' + str(stdev)+ ' ' + str(max_score))
    TFidf_scores.append(row_scores)
ideas['Slice Sim Scores']=TFidf_scores
ideas['Slice Mean Sim']=means_slice
ideas['Slice Stdev Sim']=stdevs_slice
ideas['Slice Max Score']=maxes_slice

In [ ]:
ideas['Slice Mean Sim']=means_slice
ideas['Slice Stdev Sim']=stdevs_slice
ideas['Slice Max Score']=maxes_slice
ideas['Mean Sim']=means
ideas['Stdev Sim']=stdevs
ideas['Max Score']=maxes

In [ ]:
ideas['Slice Size']=slice_sizes

In [2]:
ideas.to_csv('ideasJuly15.csv')

NameError: name 'ideas' is not defined

In [ ]:
ideas = ideas.drop(['Slice Sim Scores'], axis=1)

In [ ]:
baseline

In [ ]:
plt.scatter(ideas['Slice Mean Sim'], ideas['Slice Size'])
plt.xlabel("Mean of cosine similarity scores")
plt.ylabel("Size of slice (number of papers)")
plt.show()

In [ ]:
plt.scatter(ideas['Mean Sim'], ideas['tagsWC'])
plt.xlabel("Mean of cosine similarity scores")
plt.ylabel("Number of tags on research concepts")
plt.show()

In [ ]:
plt.scatter(ideas['Mean Sim'], ideas['abstractWC'])
plt.xlabel("Mean of cosine similarity scores")
plt.ylabel("Word count of research concepts")
plt.show()

In [ ]:
dataset__ = pandas.DataFrame(columns=list('AB'))
#for i in range(0, len(ideas['newTags'][3])):
#    for j in range(0,len(dataset)):
for i in range(0, len(dataset)):
    for j in range(0,len(dataset['HighLevel'][i])):
        if dataset['HighLevel'][i][j] == 'technology':
            print('yes!')
            df2 = pandas.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
            #print(df2)
            #df2 = pandas.DataFrame([[dataset['IdeasTagsPlusAbstract'][i], dataset['HighLevel'][i]]], columns=['IdeasTagsPlusAbstract','HighLevel'])
    dataset__.append(df2)            

In [ ]:
ideas.to_csv('ideas-results.csv')

In [ ]:
ideas

In [ ]:
ideas_ = pandas.DataFrame()
ideas_['Title']=ideas['Title']
ideas_['Mean']=ideas['Mean Sim']
ideas_['Stdev']=ideas['Stdev Sim']
ideas_['Max']=ideas['Max Score']

In [ ]:
ideas_.to_csv('ideas_scored_V2.csv')


In [ ]:
ideas = pandas.read_csv('ideas-results.csv')
ideas_ = pandas.read_csv('ideas_scored_V2.csv')
baseline = pandas.read_csv('basline.csv')

In [ ]:
ideas_

In [47]:
ideas = pandas.read_csv('ideasJuly15.csv')

In [ ]:
baseline['Acad Sim Scores'][0]

In [4]:
ideas

,Unnamed: 0,Unnamed: 0.1,Title,Description,Key Words,Terms,Categories,Stakeholders,Importance,Methdologies,...,abstractWC,tagsWC,combinedAbsandTags,hlvl,newTags,TagsPlusAbstract,Sim Scores,Mean Sim,Stdev Sim,Max Score
0,0,0,Visualization of Data About or From Tools in A...,Combining live sensor data and tools with AR d...,"connected tools, manufacturing, AR display, in...","interactive systems and tools, Internet of Thi...","Business, Technology, Use Cases, Industries","Operators of manufacturing, repair, maintenanc...",AREA members with manufacturing facilities for...,An experimental environment using a range of c...,...,502,4,interactive_systems_and_tools internet_of_thin...,business;technology;use cases;industries,"['business', 'technology', 'use cases', 'indus...",visualization data tool ar display combining l...,"[0.007068058647219471, 0.20977597990017335, 0....",0.069014,0.042070,0.296288
1,1,1,AR for Material Management and/or Supply Chain...,Many industries track supply chains and juggle...,"logistics data processing, materials managemen...","interactive systems and tools, Internet of Thi...","Business, Technology, Use Cases, End Users","operators of manufacturing, repair, employees ...",AREA members with manufacturing facilities for...,An experimental environment using a range of m...,...,422,10,interactive_systems_and_tools internet_of_thin...,business;technology;use cases;end users,"['business', 'technology', 'use cases', 'end u...",ar material management supply chain flow work ...,"[0.006817976767927102, 0.06988174099523532, 0....",0.060330,0.033708,0.216651
2,2,2,Dynamic Distribution of Sensing and Processing...,Deploying enterprise AR solutions via head-mou...,"distributed computing architecture, body-worn ...","body sensor networks, personal area networks, ...","Display Devices, Standards, Business, Technology","HMD manufacturers, AR display device designers...",Employees in AREA member companies frequently ...,"Studies of wearable sensors, low-latency body ...",...,400,7,body_sensor_networks personal_area_networks mo...,display devices;standards;business;technology,"['displays', 'standards', 'business', 'technol...",dynamic distribution sensing processing capaci...,"[0.013390898860077499, 0.047576539636666706, 0...",0.057300,0.031986,0.260657
3,3,3,Wearable vs Handheld vs Projection: Methods fo...,"Tablets, mobile phones and other handheld devi...","efficiency, handheld, projection AR, wearable ...","head worn displays, wearable computers,","Display Devices, Business","Operations leaders, financial management, OEM ...",Wearable AR displays provide frontline workers...,The research will contribute to development of...,...,344,3,head_worn_displays wearable_computers wearable...,display devices;business,"['displays', 'business']",wearable v handheld v projection method quanti...,"[0.033671177087511106, 0.08104311856430285, 0....",0.063788,0.035677,0.246399
4,4,4,Impact of AR Delivery on People Living in Mult...,"The Multidimensional Poverty Index (MPI) ""meas...","United Nations, Sustainable Development Goals ...","social aspects, social and economic effects, s...","Business, End Users",Government officials and policymakers in World...,"The Multidimensional Poverty Index (MPI) ""meas...",The proposed research would need to decide upo...,...,538,4,social_aspects social_and_economic_effects sub...,business;end users,"['business', 'end users and user experience']",impact ar delivery people living multidimensio...,"[0.010956530827323164, 0.06569176479038012, 0....",0.029010,0.018373,0.123466
5,5,5,Video Passthrough vs. See-through Optics: Impa...,Head-Mounted AR displays (HMD) blend simulated...,"Video see-through, optical passthrough, field ...","head-up displays, three-dimensional displays, ...","Display Devices, Technology, End Users","Operations leaders, HMD designers, Safety & IT...",Most of the leading AR display component and s...,Studies of human peripheral vision requirement...,...,323,9,head-up_displays thre

In [ ]:
x = ideas['Sim Scores'][0]

import ast

x = ast.literal_eval(x)


In [ ]:
data = ideas['Sim Scores'][0]
data = ast.literal_eval(data)


for i in range(1, len(ideas)):
    new = ideas['Sim Scores'][i]
    new = ast.literal_eval(new)
    data =  data + new

mu, std = norm.fit(data)

# Plot the histogram.
plt.hist(data, bins=25, density=True, alpha=0.6, color='g')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
plt.xlim([0, .35])
plt.ylim([0, 35])
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Research Topic Score"+" (mu = %.2f,  std = %.2f)" % (mu, std)

plt.title(title)
plt.show()



In [ ]:
data = baseline['Acad Sim Scores'][0]
for i in range(1, len(baseline)):
    data =  data + baseline['Acad Sim Scores'][i]
mu, sd = norm.fit(data)
my_dist = norm(mu, sd)  
my_dist.ppf(0.99)

In [ ]:
data = baseline['Fict Sim Scores'][0]
for i in range(1, len(baseline)):
    data =  data + baseline['Fict Sim Scores'][i]
mu, sd = norm.fit(data)
my_dist = norm(mu, sd)  
my_dist.ppf(0.99)

In [22]:
import ast
new_string = []
for i in range(0, len(ideas)):
    string = ideas['Sim Scores'][i]
    string = ast.literal_eval(string)
    new_string.append(string)
ideas['Sim Scores (new)'] = new_string

ValueError: malformed node or string: [0.006851578088806099, 0.21307302227805638, 0.03977012852212374, 0.023709627395366195, 0.045284647697829625, 0.0988612525237848, 0.07572226465054381, 0.019329162201095162, 0.03273659544941378, 0.15223078910911114, 0.049608502362608296, 0.021838709919226505, 0.04690442102427681, 0.05174195641498132, 0.022685053207720882, 0.044280000251840444, 0.08580631420776036, 0.06740560970586511, 0.036130694048131, 0.049912459362423015, 0.07074427439884057, 0.07750546408338455, 0.06570032891468768, 0.03824712565905814, 0.03289978471222798, 0.002264236661196018, 0.03078104389467619, 0.02600735497001829, 0.024793889175764763, 0.15328488424614586, 0.04447197138206867, 0.1157073650076432, 0.026928701415783195, 0.043998246515811475, 0.077954234554, 0.03191154876145026, 0.06013018658956315, 0.06508293354647776, 0.020151595214786308, 0.010842594124188913, 0.02211376515290379, 0.09804985483822634, 0.058203918027161225, 0.06162124343367035, 0.039796985840728945, 0.058270

In [27]:
test = ideas['Sim Scores (new)'][5]
test2 = ideas['Acad Filtered Scores'][5]
test3 = ideas['Fict Filtered Scores'][5]

In [28]:
len(test3)

4329

In [29]:
len(test)

5272

In [30]:
len(test2)

2533

In [7]:
filtered_scores = []
for j in range(0,len(test)):
    if test[j]>0.058817154045084:
        filtered_scores.append(re)
filtered_scores.append(filtered_scores)

[0.01424284389398072,
 0.07154493986180466,
 0.0441487177567972,
 0.012909533215764008,
 0.05273834568756655,
 0.09338303965094297,
 0.1332937756016214,
 0.054433094716431804,
 0.0397415258567808,
 0.042768139151868945,
 0.03840995860401587,
 0.032904129929033646,
 0.07204580532501408,
 0.07954604795101487,
 0.013771734432826643,
 0.019704056214783654,
 0.0190182086046132,
 0.042698649483074114,
 0.023080811900480176,
 0.022040151222011726,
 0.08194684097299576,
 0.10412998570117844,
 0.06517058305646915,
 0.04696397700565766,
 0.03037443944319692,
 0.006460418760413145,
 0.037410261725041166,
 0.03370879759139173,
 0.03304026110257259,
 0.09668270220215787,
 0.033282131985852434,
 0.028967347665285668,
 0.04112988768941648,
 0.03615857563972115,
 0.07344559301787522,
 0.017969248685171312,
 0.07157029602929053,
 0.02212763158834296,
 0.03409851181915652,
 0.023461543173931235,
 0.04202299754278347,
 0.07992970888318351,
 0.06222957690973502,
 0.055341972168081906,
 0.04209004322040912

In [86]:
filtered_scores = []
for i in range (0, len(ideas)):
    filtered_scores_ = []
    score = ideas['Sim Scores'][i]
    for j in range(0,len(score)):
        if score[j]>0.058817154045084:
            filtered_scores_.append(score[j])
    filtered_scores.append(filtered_scores_)
ideas['Acad Filtered Scores']=filtered_scores

In [87]:
filtered_scores = []
for i in range (0, len(ideas)):
    filtered_scores_ = []
    score = ideas['Sim Scores'][i]
    for j in range(0,len(score)):
        if score[j]>0.03083764926999431:
            filtered_scores_.append(score[j])
    filtered_scores.append(filtered_scores_)
ideas['Fict Filtered Scores']=filtered_scores

In [88]:
points = []
for i in range(0,len(ideas)):
    scores = ideas['Acad Filtered Scores'][i]
    point = 0
    for j in range(0,len(scores)):
        score = scores[j]
        if score<0.075:
            point=point+1
        if score<0.1 and score>=0.075:
            point=point+2
        if score<0.125 and score>=0.1:
            point=point+3
        if score<0.15 and score>=0.125:
            point=point+4
        if score<0.175 and score>=0.15:
            point=point+5
        if score<0.2 and score>=0.175:
            point=point+6
        if score<0.225 and score>=0.2:
            point=point+7
        if score<0.25 and score>=0.225:
            point=point+8
        if score<0.275 and score>=0.25:
            point=point+9
        if score>=0.275:
            point=point+10
    points.append(point)
ideas['Acad Points'] = points
        




In [89]:
ideas['Score'] = round(5*ideas['Acad Points']/max(ideas['Acad Points']),2)

In [90]:
ideas.to_csv('newScoresForIdeas.csv')

In [91]:
relevant_citations = []
for j in range(0,len(ideas)):
    x = ideas['Sim Scores'][j]    
    temp = pandas.DataFrame()
    temp['scores'] = x
    temp = temp.sort_values('scores',ascending=False)
    index = temp[:5].index.values
    papers = ''
    for i in range(0,len(index)):
        paper = dataset['Citation'][index[i]]
        if papers == '':
            papers = paper
        if papers != '':
            papers = papers + ' | ' + paper
    relevant_citations.append(papers)
    #print(dataset['Citation'][index[i]])
ideas['Relevant Publications']=relevant_citations

In [92]:
ideas.to_csv('newScoresForIdeas.csv')

In [93]:
score_research_topcs = pandas.DataFrame()
score_research_topcs['Title']=ideas['Title']
score_research_topcs['Description']=ideas['Description']
score_research_topcs['Keywords']=ideas['Keywords']
score_research_topcs['Categories']=ideas['Categories']
score_research_topcs['Stakeholders']=ideas['Stakeholders']
score_research_topcs['Importance']=ideas['Importance']
score_research_topcs['Methodologies']=ideas['Methdologies']
score_research_topcs['Timeframes']=ideas['Timeframes']
score_research_topcs['Program']=ideas['Program']
score_research_topcs['Miscellaneous']=ideas['Miscellaneous']
score_research_topcs['Author']=ideas['Author']
score_research_topcs['History']=ideas['History']
score_research_topcs['Score']=ideas['Score']
score_research_topcs['Relevant Publications']=ideas['Relevant Publications']


score_research_topcs.to_csv('scored_topicsAug24.csv')

In [ ]:
ideas['Sim Scores'] = new_string

In [46]:
ideas.drop(columns=['Sim Scores','Sim Scores (new)','Acad Filtered Scores','Fict Filtered Scores'])
ideas.to_csv('ideas_final2.csv')

In [ ]:
from scipy.stats import norm
mu = np.mean(data)
std = np.std(data)
cdf = norm.cdf(data,loc=mu,scale=std)

In [ ]:
cdf_val = []
for i in range(0,len(cdf)):
    cdf_val.append(cdf[i])

In [ ]:
cdf_val

In [ ]:
cdf_values = pandas.DataFrame()
cdf_values['academic scores'] = data
cdf_values['cdf'] = cdf_val

In [ ]:
norm.ppf(cdf(0.99))

In [ ]:
counter = 0
for i in range(0,len(cdf)):
    if cdf[i] > 0.99:
        counter=counter+1
print(counter)


In [ ]:
list1 = cdf.tolist()

In [ ]:
cdf.size

In [ ]:
# defining the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
  
# No of Data points
N = len(data)
  
# initializing random values
#data = np.random.randn(N)
  
# getting data of the histogram
count, bins_count = np.histogram(data, bins=10)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
  
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)
  
# plotting PDF and CDF
plt.plot(bins_count[1:], pdf, color="red", label="PDF")
plt.plot(bins_count[1:], cdf, label="CDF")
plt.legend()

In [ ]:
cdf

In [ ]:
data = baseline['Acad Sim Scores'][0]
for i in range(1, len(baseline)):
    data =  data + baseline['Acad Sim Scores'][i]

mu, std = norm.fit(data)

# Plot the histogram.
plt.hist(data, bins=25, density=True, alpha=0.6, color='g')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
plt.xlim([0, .35])
plt.ylim([0, 35])
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Academic"+" (mu = %.2f,  std = %.2f)" % (mu, std)

plt.title(title)
plt.show()

data = baseline['Fict Sim Scores'][0]
for i in range(1, len(baseline)):
    data =  data + baseline['Fict Sim Scores'][i]

mu, std = norm.fit(data)

# Plot the histogram.
plt.hist(data, bins=25, density=True, alpha=0.6, color='g')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
plt.xlim([0, .35])
plt.ylim([0, 35])
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Fiction"+" (mu = %.2f,  std = %.2f)" % (mu, std)

plt.title(title)
plt.show()

In [ ]:
from scipy.stats import norm
# Fit a normal distribution to the data:
for i in range(0,len(ideas)):
    data = ideas['Sim Scores'][i]
    mu, std = norm.fit(data)

# Plot the histogram.
    plt.hist(data, bins=25, density=True, alpha=0.6, color='g')

# Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    plt.xlim([0, .35])
    plt.ylim([0, 35])
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)
    title = "RC #"+str(i)+" (mu = %.2f,  std = %.2f)" % (mu, std)

    plt.title(title)

    plt.show()

In [ ]:

TFidf_scores = []
#int(TFidf_scores)
for i in range(0,len(dataset)):
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(dataset['cleanAbstract'][i], dataset['cleanAbstract'][j])
        row_scores.append(score)
        #rint(score)
        #TFidf_scores[i,j].astype(float)
        #Tfidf_scores[i,j] = score
    TFidf_scores.append(row_scores)


In [ ]:
top_20 = []
for i in range(len(TFidf_scores)):
    a = TFidf_scores[i]
    l = OrderedDict(sorted(enumerate(a), key=itemgetter(1),reverse=True))
    d = dict(sorted(l.items(), key=lambda x: x[1], reverse=True)[1:21])
    keys_list = list(d)
    top_20.append(keys_list)
    
dataset["Top20Abs"] = top_20    

In [ ]:
TFidf_scores = []
#int(TFidf_scores)
for i in range(0,len(dataset)):
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(dataset['combinedTagsNew'][i], dataset['combinedTagsNew'][j])
        row_scores.append(score)
        #rint(score)
        #TFidf_scores[i,j].astype(float)
        #Tfidf_scores[i,j] = score
    TFidf_scores.append(row_scores)
    #print(i)

top_20 = []
for i in range(len(TFidf_scores)):
    a = TFidf_scores[i]
    l = OrderedDict(sorted(enumerate(a), key=itemgetter(1),reverse=True))
    d = dict(sorted(l.items(), key=lambda x: x[1], reverse=True)[1:21])
    keys_list = list(d)
    top_20.append(keys_list)

dataset["Top20Tags"] = top_20

In [ ]:
TFidf_scores = []
#int(TFidf_scores)
for i in range(0,len(dataset)):
    row_scores =[]
    for j in range(0,len(dataset)):
        score = cosine_sim(dataset['TagsPlusAbstract'][i], dataset['TagsPlusAbstract'][j])
        row_scores.append(score)
        #rint(score)
        #TFidf_scores[i,j].astype(float)
        #Tfidf_scores[i,j] = score
    TFidf_scores.append(row_scores)
    print(i)

top_20 = []
for i in range(len(TFidf_scores)):
    a = TFidf_scores[i]
    l = OrderedDict(sorted(enumerate(a), key=itemgetter(1),reverse=True))
    d = dict(sorted(l.items(), key=lambda x: x[1], reverse=True)[1:21])
    keys_list = list(d)
    top_20.append(keys_list)

dataset["Top20AbsAndTags"] = top_20

In [ ]:
dataset['Top20Tags'] = top_20
dataset["Top20Abs"] = top_20 

In [ ]:
dataset.to_csv("newdataFeb12.csv")

In [ ]:
dataset = pandas.read_csv('newDATA.csv')

In [ ]:
df_json = pandas.DataFrame()
df_json['Index']=dataset['Unnamed: 0']
df_json['Title']=dataset['Title']
df_json['Author']=dataset['newNames']
#df_json['Issue date']=dataset['Issue date']
df_json['Publication year']=dataset['Publication year']
df_json['DOI']=dataset['DOI']
#df_json['Conference name']=dataset['Conference name']
df_json['Venue']=dataset['Venue']
df_json['Abstract']=dataset['Abstract']
df_json['LowLevel']=dataset['lowlvl']
df_json['MidLevel']=dataset['midlvl']
df_json['HighLevel']=dataset['highlvl']
df_json['Top20AbsAndTags']=dataset['Top20AbsAndTags']
#df_json['Top20Abs']=dataset['Top20Abs']
#df_json['Top20Tags']=dataset['Top20Tags']
df_json['Citation']=dataset['Citation']

df_json=df_json.sort_values(by='Publication year', ascending=False)

In [ ]:
df_json

In [ ]:
for j in range(0,len(df_json)):
    a=df_json['Top20Abs'][j]
    a=re.sub("'","",a)
    a=re.sub("\[","",a)
    a=re.sub("\]","",a)
    a=a.split(', ')
    for i in range(0, len(a)): 
        a[i] = int(a[i]) 
    df_json['Top20Abs'][j] = a


In [ ]:
for j in range(0,len(df_json)):
    a=df_json['Top20Tags'][j]
    a=re.sub("'","",a)
    a=re.sub("\[","",a)
    a=re.sub("\]","",a)
    a=a.split(', ')
    for i in range(0, len(a)): 
        a[i] = int(a[i]) 
    df_json['Top20Tags'][j] = a


In [ ]:
for j in range(0,len(df_json)):
    a=df_json['Top20AbsAndTags'][j]
    a=re.sub("'","",a)
    a=re.sub("\[","",a)
    a=re.sub("\]","",a)
    a=a.split(', ')
    for i in range(0, len(a)): 
        a[i] = int(a[i]) 
    df_json['Top20AbsAndTags'][j] = a

In [ ]:
df_json

In [ ]:
with open('low_terms.txt', 'w') as f:
    for item in varnames_low:
        f.write("%s\n" % item)

In [ ]:
df = dataset.copy()
df['LowLevel']=df['lowlvl']
df['MidLevel']=df['midlvl']
new_df = df.drop(columns=['lowlvl', 'concat_test','midlvl','combinedTagsNew','Unnamed: 0.1','combinedTags','cleanAbstract','TagsPlusAbstract','Unnamed: 0'])

In [ ]:
result = df_json.to_json(orient="records")
#parsed = json.loads(result)
#json.dumps(parsed, indent=4) 

with open('area-research-data.json', 'w') as fp:
    print(result, file=fp)
    #json.dump(result, fp, indent=4)
#with open('data.json', 'w') as outfile:
#    json.dump(result, outfile)

In [ ]:
a = df_json.to_json(orient="records")

In [ ]:
df_json.to_csv('df_json.csv')

In [ ]:
lowlvl = []
for i in range(0, len(dataset)):
    terms = str(dataset['LowLevel'][i])
    terms = re.split(';', terms)
    lowlvl.append(terms)

In [ ]:
AllLowLvlTerms = []
for i in range(0,len(lowlvl)):
    for j in range(0, len(lowlvl[i])):
        text = lowlvl[i][j]
        AllLowLvlTerms.append(text)

In [ ]:
b= list(set(AllLowLvlTerms))

In [ ]:
df = pandas.DataFrame()
df['LowLevel']=b
counter = len(df)
freq_count = []
mid_levels = []

for i in range(0, len(df['LowLevel'])):
    string1 = df['LowLevel'][i]
    for j in range(0, len(alias['LowLevel'])):
        string2 = alias['LowLevel'][j]
        if string1==string2:
            mid_levels.append(alias['MidLevel'][j])

df['MidLevel']=mid_levels
df.to_csv('AllLowLvlTerms.csv')           


In [ ]:
for i in range(0,len(df)):
    counter = counter-1
    freq = 0;
    for j in range(0,len(dataset)):
        test_string = dataset["LowLevel"][j]
        #res = isinstance(test_string, str) 
        test = dataset['LowLevel'][j].split(';')
        for k in range(0,len(test)):
            if df["LowLevel"][i] == test[k]:
                freq = freq+1
    freq_count.append(freq)
    print(counter)
    
#df['MidLevel']=mid_levels
df['freq']=freq_count

df.to_csv('AllLowLvlTerms.csv')

In [ ]:
midlvl = lowlvl
counter = 0
for j in range(0, len(alias)):
    counter = counter + 1
    for i in range(0, len(lowlvl)):
        for k in range(0,len(lowlvl[i])):
            string1 = alias['LowLevel'][j]
            string2 = lowlvl[i][k]
            if (string1 == string2):
                midlvl[i][k]=alias['MidLevel'][j]
    print(counter)              
                

In [ ]:
for i in range(0, len(midlvl)):
    midlvl[i] = list(set(midlvl[i]))


In [ ]:
dataset['MidLevel']=midlvl
new_df['Midlevel']

In [ ]:
for i in range(0,len(midlvl)):
    for j in range(0,len(midlvl[i])):
        counter = 0
        for k in range(0,len(midlvl_names)):
            string1 = midlvl_names[k]
            string2 = midlvl[i][j]
            if string1==string2:
                counter=counter+1
        if counter == 0:
            print(midlvl[i][j])



In [ ]:
lowlevel = dataset["LowLevel"]

In [ ]:
lowlevel = []
for i in range(0,len(dataset)):
    full = dataset["LowLevel"][i].split(';')
    lowlevel.append(full)
    
    

In [ ]:
data = {}
data_row = pandas.DataFrame()
data_row['category']=varnames    
#for j in range (0,len(table)):
#    if table[j,0] != 0:
#        data_row[varnames[j]]=table[j,0]        

In [ ]:
data ={}
data['']=[]
for i in range (0, len(table)):
    data[''].append({
        "category":varnames[i]        
    })
    for j in range (0, len(table)):
        if table[j,0] != 0:
            data[''].append(
                {varnames[j]:table[j,0]}
            )

In [ ]:
file1 = open("low-test.json","w") 
file1.write('[')
for i in range (0, len(table_low)):
    file1.write("{"+'"category"'+":"+'"'+varnames_low[i]+'",')
    for j in range(0,len(table_low)):
        if table_low[i,j] != 0:
            file1.write('"'+varnames_low[j]+'"'+': '+'"'+str(table_low[i,j])+'",')
    file1.write("},")
file1.write('{"category": ""}')
file1.write(']')

In [ ]:
file1 = open("low.csv","w")
file1.write("Source,Target,Freq\n")
for i in range(0, len(table_low)):
    for j in range(0,len(table_low)):
        if table_low[i,j] != 0:
            file1.write(varnames_low[i]+","+varnames_low[j]+","+str(table_low[i,j])+"\n")

In [ ]:
file1 = open("mid.json","w") 
file1.write('[')
for i in range (0, len(table_mid)):
    file1.write("{"+'"category"'+":"+'"'+varnames_mid[i]+'",')
    for j in range(0,len(table_mid)):
        if table_mid[i,j] != 0:
            file1.write('"'+varnames_mid[j]+'"'+': '+'"'+str(table_mid[i,j])+'",')
    file1.write("},")
file1.write('{"category": ""}')
file1.write(']')

In [ ]:
data={}
data={"category":varnames[0]}
for i in range (0,len(varnames)):
    if table[i,0] != 0:
        data.append({varnames[i]:table[i,0]})

In [ ]:
all_lows =[]
for i in range(0, len(df_json)):
    terms = df_json["LowLevel"][i].split(';')
    for j in range (0, len(terms)):
        all_lows.append(terms[j])
a = list(set(all_lows))
all_lows=a
print(len(all_lows))

In [ ]:
new_alias=new_alias.sort_values(by='LowLevel', ascending=True)

In [ ]:
new_alias_ = pandas.read_csv('new_alias_no_underscores.csv')

In [ ]:
new_alias_=new_alias_.sort_values(by='LowLevel', ascending=True)
file1 = open("terms_new.json","w") 
file1.write('[')
for i in range (0, len(new_alias_)):
    file1.write("{"+'"label"'+":"+'"'+new_alias_['LowLevel'][i]+'",')
    file1.write('"category"'+":"+'"'+new_alias_['MidLevel'][i]+'"')
    file1.write("},")
#file1.write('{"category": ""}')
file1.write(']')

In [ ]:
#varnames carries all midlevels
buckets = []
for i in range (0, len(varnames_mid)):
    mid_string = varnames_mid[i]
    terms = []
    for j in range(0, len(new_alias_)):
        if new_alias_['MidLevel'][j] == str(mid_string+"_"):
            terms.append(new_alias_['LowLevel'][j])
    buckets.append(terms)
            

In [ ]:
file1 = open("topics.json","w") 
file1.write('[')
for i in range (0, len(varnames_mid)):
    file1.write("{"+'"term"'+":"+'"'+varnames_mid[i]+'",')    
    file1.write('"lowLevel"'+":"+'[')
    for j in range(0,len(buckets[i])):
        file1.write('"'+str(buckets[i][j])+'",')
    file1.write("]")    
    file1.write("},")
#file1.write('{"category": ""}')
file1.write(']')